In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install boto3 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 67.7 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.23
    Uninstalling botocore-1.35.23:
      Successfully uninstalled botocore-1.35.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.1 requires botocore<1.35.24,>=1.35.16, but you have botocore 1.29.165 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers datasets faiss-cpu boto3 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 49.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.

In [5]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install langchain_together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 9.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 3.5 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np
import faiss
import pandas as pd
from langchain.chains import LLMChain
from langchain_together import ChatTogether
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document

# Initialize your LLM model
chat_model = ChatTogether(
    together_api_key="c51c9bcaa6bf7fae3ce684206311564828c13fa2e91553f915fee01d517ccee9",
    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
)

# Prompt engineering: Make the context more specific and improve the instructions
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are an expert financial advisor. Use the context to answer questions accurately and concisely.\n"
        "Context: {context}\n\n"
        "Question: {question}\n\n"
        "Answer (be specific and avoid hallucinations):"
    )
)

# Load the embedding model
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Example dummy text (can be replaced with actual data)
dummy_text = ["""
Bajaj customers: Interest rates range from 7.16% to 8.40% per annum (p.a.)
- Senior citizens: Interest rates range from 7.39% to 8.65% p.a., with an additional rate benefit of up to 0.40% p.a.
- Non-Resident Indians (NRIs): Interest rates of up to 8.35% p.a.
- Systematic Deposit Plan (SDP): A regular monthly deposit scheme
Bajaj Finance FDs are considered to be safe and secure, with an AAA(Stable) rating from ICRA and AAA/STABLE from CRISIL.
The minimum investment amount is Rs. 15,000, and the tenure can range from 12 to 60 months.
Before booking an FD, it's important to check the eligibility criteria and document requirements on the lender's website.
"""]

# Create embeddings
embeddings = embedding_model.embed_documents(dummy_text)
embeddings = np.array(embeddings)
dimension = embeddings.shape[1]

# Initialize FAISS index and add embeddings
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Setup the docstore and mapping between index and documents
docstore = InMemoryDocstore({0: Document(page_content=dummy_text[0])})
index_to_docstore_id = {0: 0}

# Initialize the FAISS vector store
vector_store = FAISS(
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_model.embed_query  # Use the embedding function
)

# Define the QA chain for answering questions
qa_chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template
)

# Example usage
def get_answer(question):
    # Create an embedding for the query
    query_embedding = embedding_model.embed_query(question)
    
    # Search the vector store for the most similar documents
    D, I = index.search(np.array([query_embedding]), k=1)
    
    # Retrieve the most relevant document
    doc_id = I[0][0]
    document = docstore.search(doc_id)
    context = document.page_content
    
    # Use the qa_chain to get an answer
    answer = qa_chain.run(context=context, question=question, clean_up_tokenization_spaces=False)
    
    return answer

if __name__ == "__main__":
    # Define a test question
    question = "What is the interest rate for senior citizens for FD?"
    
    # Call get_answer and print the result
    answer = get_answer(question)
    print("Answer:", answer)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Answer: For senior citizens, the interest rate for FD ranges from 7.39% to 8.65% per annum, with an additional rate benefit of up to 0.40% per annum.
